In [2]:
import pandas as pd
df = pd.read_csv(r"C:\Users\astle\Downloads\study_plan_dataset.csv")
print(df.shape)
print(df.columns)
print(df.head())
print(df.isnull().sum())  


(1000, 9)
Index(['Student_ID', 'GPA', 'Learning_Style', 'Available_Hours_Per_Day',
       'Free_Days_Per_Week', 'Subject_Count', 'Exam_Priority',
       'Weak_Subjects_Count', 'Recommended_Hours_Per_Subject'],
      dtype='object')
   Student_ID   GPA Learning_Style  Available_Hours_Per_Day  \
0           1  5.63    Kinesthetic                        3   
1           2  6.70       Auditory                        7   
2           3  9.19       Auditory                        2   
3           4  5.44       Auditory                        8   
4           5  7.45       Auditory                        2   

   Free_Days_Per_Week  Subject_Count Exam_Priority  Weak_Subjects_Count  \
0                   7              4           Low                    1   
1                   4              4           Low                    1   
2                   7              7           Low                    2   
3                   6              8        Medium                    3   
4             

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

X = df.drop(columns=["Student_ID", "Recommended_Hours_Per_Subject"])
y = df["Recommended_Hours_Per_Subject"]

categorical_cols = ["Learning_Style", "Exam_Priority"]
numeric_cols = [c for c in X.columns if c not in categorical_cols]

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numeric_cols),
    ("cat", OneHotEncoder(sparse_output=False, handle_unknown="ignore"), categorical_cols)
])

X_processed = preprocessor.fit_transform(X)


In [9]:
import joblib
joblib.dump(preprocessor, "preprocessor.joblib")


['preprocessor.joblib']

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y.values, test_size=0.2, random_state=42
)


In [16]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

def build_model(input_dim):
    model = Sequential([
        Dense(256, activation="relu", input_shape=(input_dim,)),
        BatchNormalization(),
        Dropout(0.3),

        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(0.25),

        Dense(64, activation="relu"),
        BatchNormalization(),
        Dropout(0.2),

        Dense(32, activation="relu"),
        Dense(1, activation="linear")
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss="mse",
                  metrics=["mae"])
    return model

input_dim = X_train.shape[1]
model = build_model(input_dim)
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 256)            │         3,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 48,385 (189.00 KB)

 Trainable params: 47,489 (185.50 KB)

 Non-trainable params: 896 (3.50 KB)

In [19]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

callbacks = [
    EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True, verbose=1),
    ModelCheckpoint("study_plan_model.h5", save_best_only=True, monitor="val_loss", verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=8, verbose=1)
]

history = model.fit(
    X_train, y_train,
    validation_split=0.15,
    epochs=200,
    batch_size=32,
    callbacks=callbacks,
    verbose=2
)


Epoch 1/200

Epoch 1: val_loss improved from None to 0.52599, saving model to study_plan_model.h5


22/22 - 3s - 116ms/step - loss: 0.9198 - mae: 0.7479 - val_loss: 0.5260 - val_mae: 0.5696 - learning_rate: 0.0010
Epoch 2/200

Epoch 2: val_loss improved from 0.52599 to 0.47142, saving model to study_plan_model.h5


22/22 - 0s - 11ms/step - loss: 0.4685 - mae: 0.5392 - val_loss: 0.4714 - val_mae: 0.5155 - learning_rate: 0.0010
Epoch 3/200

Epoch 3: val_loss improved from 0.47142 to 0.42496, saving model to study_plan_model.h5


22/22 - 0s - 14ms/step - loss: 0.3327 - mae: 0.4537 - val_loss: 0.4250 - val_mae: 0.4836 - learning_rate: 0.0010
Epoch 4/200

Epoch 4: val_loss improved from 0.42496 to 0.34607, saving model to study_plan_model.h5


22/22 - 0s - 19ms/step - loss: 0.2636 - mae: 0.4006 - val_loss: 0.3461 - val_mae: 0.4224 - learning_rate: 0.0010
Epoch 5/200

Epoch 5: val_loss improved from 0.34607 to 0.28392, saving model to study_plan_model.h5


...

22/22 - 0s - 13ms/step - loss: 0.0332 - mae: 0.1435 - val_loss: 0.0099 - val_mae: 0.0775 - learning_rate: 5.0000e-04
Epoch 54/200

Epoch 54: val_loss did not improve from 0.00991
22/22 - 0s - 10ms/step - loss: 0.0427 - mae: 0.1579 - val_loss: 0.0110 - val_mae: 0.0835 - learning_rate: 5.0000e-04
Epoch 55/200

Epoch 55: val_loss did not improve from 0.00991
22/22 - 0s - 5ms/step - loss: 0.0378 - mae: 0.1452 - val_loss: 0.0105 - val_mae: 0.0827 - learning_rate: 5.0000e-04
Epoch 56/200

Epoch 56: val_loss did not improve from 0.00991
22/22 - 0s - 6ms/step - loss: 0.0460 - mae: 0.1645 - val_loss: 0.0116 - val_mae: 0.0838 - learning_rate: 5.0000e-04
Epoch 57/200

Epoch 57: val_loss did not improve from 0.00991
22/22 - 0s - 6ms/step - loss: 0.0346 - mae: 0.1423 - val_loss: 0.0104 - val_mae: 0.0799 - learning_rate: 5.0000e-04
Epoch 58/200

Epoch 58: val_loss did not improve from 0.00991
22/22 - 0s - 5ms/step - loss: 0.0389 - mae: 0.1541 - val_loss: 0.0108 - val_mae: 0.0801 - learning_rate: 5.0

22/22 - 0s - 12ms/step - loss: 0.0331 - mae: 0.1386 - val_loss: 0.0094 - val_mae: 0.0753 - learning_rate: 5.0000e-04
Epoch 60/200

Epoch 60: val_loss did not improve from 0.00938
22/22 - 0s - 8ms/step - loss: 0.0346 - mae: 0.1390 - val_loss: 0.0096 - val_mae: 0.0768 - learning_rate: 5.0000e-04
Epoch 61/200

Epoch 61: val_loss did not improve from 0.00938
22/22 - 0s - 9ms/step - loss: 0.0347 - mae: 0.1413 - val_loss: 0.0136 - val_mae: 0.0901 - learning_rate: 5.0000e-04
Epoch 62/200

Epoch 62: val_loss improved from 0.00938 to 0.00849, saving model to study_plan_model.h5


22/22 - 0s - 12ms/step - loss: 0.0323 - mae: 0.1369 - val_loss: 0.0085 - val_mae: 0.0745 - learning_rate: 5.0000e-04
Epoch 63/200

Epoch 63: val_loss did not improve from 0.00849
22/22 - 0s - 7ms/step - loss: 0.0360 - mae: 0.1430 - val_loss: 0.0103 - val_mae: 0.0807 - learning_rate: 5.0000e-04
Epoch 64/200

Epoch 64: val_loss did not improve from 0.00849
22/22 - 0s - 6ms/step - loss: 0.0403 - mae: 0.1518 - val_loss: 0.0114 - val_mae: 0.0837 - learning_rate: 5.0000e-04
Epoch 65/200

Epoch 65: val_loss did not improve from 0.00849
22/22 - 0s - 6ms/step - loss: 0.0314 - mae: 0.1338 - val_loss: 0.0099 - val_mae: 0.0790 - learning_rate: 5.0000e-04
Epoch 66/200

Epoch 66: val_loss did not improve from 0.00849
22/22 - 0s - 7ms/step - loss: 0.0306 - mae: 0.1294 - val_loss: 0.0119 - val_mae: 0.0855 - learning_rate: 5.0000e-04
Epoch 67/200

Epoch 67: val_loss did not improve from 0.00849
22/22 - 0s - 6ms/step - loss: 0.0376 - mae: 0.1484 - val_loss: 0.0117 - val_mae: 0.0829 - learning_rate: 5.00

22/22 - 0s - 9ms/step - loss: 0.0340 - mae: 0.1388 - val_loss: 0.0082 - val_mae: 0.0711 - learning_rate: 2.5000e-04
Epoch 75/200

Epoch 75: val_loss did not improve from 0.00820
22/22 - 0s - 7ms/step - loss: 0.0365 - mae: 0.1426 - val_loss: 0.0099 - val_mae: 0.0751 - learning_rate: 2.5000e-04
Epoch 76/200

Epoch 76: val_loss did not improve from 0.00820
22/22 - 0s - 8ms/step - loss: 0.0353 - mae: 0.1396 - val_loss: 0.0114 - val_mae: 0.0808 - learning_rate: 2.5000e-04
Epoch 77/200

Epoch 77: val_loss improved from 0.00820 to 0.00797, saving model to study_plan_model.h5


22/22 - 0s - 12ms/step - loss: 0.0344 - mae: 0.1354 - val_loss: 0.0080 - val_mae: 0.0696 - learning_rate: 2.5000e-04
Epoch 78/200

Epoch 78: val_loss did not improve from 0.00797
22/22 - 0s - 7ms/step - loss: 0.0287 - mae: 0.1267 - val_loss: 0.0102 - val_mae: 0.0769 - learning_rate: 2.5000e-04
Epoch 79/200

Epoch 79: val_loss did not improve from 0.00797
22/22 - 0s - 7ms/step - loss: 0.0346 - mae: 0.1383 - val_loss: 0.0099 - val_mae: 0.0758 - learning_rate: 2.5000e-04
Epoch 80/200

Epoch 80: val_loss did not improve from 0.00797
22/22 - 0s - 7ms/step - loss: 0.0308 - mae: 0.1315 - val_loss: 0.0101 - val_mae: 0.0764 - learning_rate: 2.5000e-04
Epoch 81/200

Epoch 81: val_loss did not improve from 0.00797
22/22 - 0s - 7ms/step - loss: 0.0300 - mae: 0.1328 - val_loss: 0.0094 - val_mae: 0.0735 - learning_rate: 2.5000e-04
Epoch 82/200

Epoch 82: val_loss did not improve from 0.00797
22/22 - 0s - 7ms/step - loss: 0.0339 - mae: 0.1385 - val_loss: 0.0107 - val_mae: 0.0772 - learning_rate: 2.50

In [24]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
y_pred = model.predict(X_test).flatten()
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE={mse:.4f}, MAE={mae:.4f}, R2={r2:.4f}")


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
MSE=0.0076, MAE=0.0656, R2=0.9514


In [22]:
model.save("study_plan_model.h5")  
joblib.dump(preprocessor, "preprocessor.joblib")


['preprocessor.joblib']

In [30]:
import joblib
import pandas as pd
from tensorflow.keras.models import load_model
preprocessor = joblib.load("preprocessor.joblib")
model = load_model("study_plan_model.h5", compile=False)
new_student = pd.DataFrame([{
    "GPA": 8.2,
    "Learning_Style": "Visual",
    "Available_Hours_Per_Day": 5,
    "Free_Days_Per_Week": 6,
    "Subject_Count": 6,
    "Exam_Priority": "High",
    "Weak_Subjects_Count": 2
}])
X_new = preprocessor.transform(new_student)
predicted_hours = model.predict(X_new).flatten()[0]
print("Predicted hours per subject:", round(predicted_hours, 2))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
Predicted hours per subject: 1.01


In [6]:
import pandas as pd
def generate_weekly_schedule(predicted_hours_per_subject, subject_names, available_hours_per_day, 
                            free_days_per_week, weak_subjects, weak_extra=0.5, session_granularity=0.5):
    import pandas as pd
    import numpy as np
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    study_days = days[:(7-free_days_per_week)]
    rows = []
    for day in study_days:
        for subject, hours in zip(subject_names, predicted_hours_per_subject):
            # Add extra time for weak subjects
            if isinstance(weak_subjects, int) and subject_names.index(subject) < weak_subjects:
                hours += weak_extra
            elif isinstance(weak_subjects, list) and subject in weak_subjects:
                hours += weak_extra
            hours = round(hours / session_granularity) * session_granularity
            
            rows.append({
                'Day': day,
                'Subject': subject,
                'Hours': hours
            })
    
    df_detail = pd.DataFrame(rows)
    df_summary = df_detail.groupby('Day').agg({'Hours': 'sum'}).reset_index()
    total_hours = df_detail['Hours'].sum()
    target_hours = sum(predicted_hours_per_subject) * len(study_days)
    remaining = target_hours - total_hours
    
    return df_detail, df_summary, remaining, target_hours
new_student = pd.DataFrame({
    "Available_Hours_Per_Day": [4],  
    "Free_Days_Per_Week": [5],       
    "Weak_Subjects_Count": [2]       
})
predicted_hours = [2, 3, 4, 2, 1, 2]  
subject_names = ["Math","Physics","CS","Chemistry","English","Biology"]  
available_hours_per_day = int(new_student.loc[0, "Available_Hours_Per_Day"])
free_days_per_week = int(new_student.loc[0, "Free_Days_Per_Week"])
weak_subjects = int(new_student.loc[0, "Weak_Subjects_Count"])  

df_detail, df_summary, remaining, target_hours = generate_weekly_schedule(
    predicted_hours_per_subject=predicted_hours,
    subject_names=subject_names,
    available_hours_per_day=available_hours_per_day,
    free_days_per_week=free_days_per_week,
    weak_subjects=weak_subjects,
    weak_extra=0.5,            
    session_granularity=0.5    
)

display(df_summary)   
display(df_detail)    

df_summary.to_csv("weekly_schedule_summary.csv", index=False)
df_detail.to_csv("weekly_schedule_detailed.csv", index=False)

,Day,Hours
0,Monday,15.0
1,Tuesday,15.0


,Day,Subject,Hours
0,Monday,Math,2.5
1,Monday,Physics,3.5
2,Monday,CS,4.0
3,Monday,Chemistry,2.0
4,Monday,English,1.0
5,Monday,Biology,2.0
6,Tuesday,Math,2.5
7,Tuesday,Physics,3.5
8,Tuesday,CS,4.0
9,Tuesday,Chemistry,2.0


In [2]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

df = pd.read_csv(r"C:\Users\astle\Downloads\study_plan_dataset.csv")

X = df.drop(columns=["Student_ID", "Recommended_Hours_Per_Subject"])
y = df["Recommended_Hours_Per_Subject"]

categorical_cols = ["Learning_Style", "Exam_Priority"]
numeric_cols = [c for c in X.columns if c not in categorical_cols]

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numeric_cols),
    ("cat", OneHotEncoder(sparse_output=False, handle_unknown="ignore"), categorical_cols)
])

X_processed = preprocessor.fit_transform(X)
joblib.dump(preprocessor, "preprocessor.joblib")

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y.values, test_size=0.2, random_state=42
)

def build_model(input_dim):
    model = Sequential([
        Dense(256, activation="relu", input_shape=(input_dim,)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(0.25),
        Dense(64, activation="relu"),
        BatchNormalization(),
        Dropout(0.2),
        Dense(32, activation="relu"),
        Dense(1, activation="linear")
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss="mse",
                  metrics=["mae"])
    return model

input_dim = X_train.shape[1]
model = build_model(input_dim)

callbacks = [
    EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True, verbose=1),
    ModelCheckpoint("study_plan_model.h5", save_best_only=True, monitor="val_loss", verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=8, verbose=1)
]

history = model.fit(
    X_train, y_train,
    validation_split=0.15,
    epochs=200,
    batch_size=32,
    callbacks=callbacks,
    verbose=2
)

y_pred = model.predict(X_test).flatten()
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

model.save("study_plan_model.h5")
joblib.dump(preprocessor, "preprocessor.joblib")

preprocessor = joblib.load("preprocessor.joblib")
model = load_model("study_plan_model.h5", compile=False)

new_student = pd.DataFrame([{
    "GPA": 8.2,
    "Learning_Style": "Visual",
    "Available_Hours_Per_Day": 5,
    "Free_Days_Per_Week": 6,
    "Subject_Count": 6,
    "Exam_Priority": "High",
    "Weak_Subjects_Count": 2
}])

X_new = preprocessor.transform(new_student)
predicted_hours = model.predict(X_new).flatten()[0]

import numpy as np

def generate_weekly_schedule(predicted_hours_per_subject, subject_names, available_hours_per_day, 
                            free_days_per_week, weak_subjects, weak_extra=0.5, session_granularity=0.5):
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    study_days = days[:(7-free_days_per_week)]
    rows = []
    for day in study_days:
        for subject, hours in zip(subject_names, predicted_hours_per_subject):
            if isinstance(weak_subjects, int) and subject_names.index(subject) < weak_subjects:
                hours += weak_extra
            elif isinstance(weak_subjects, list) and subject in weak_subjects:
                hours += weak_extra
            hours = round(hours / session_granularity) * session_granularity
            rows.append({'Day': day, 'Subject': subject, 'Hours': hours})
    df_detail = pd.DataFrame(rows)
    df_summary = df_detail.groupby('Day').agg({'Hours': 'sum'}).reset_index()
    total_hours = df_detail['Hours'].sum()
    target_hours = sum(predicted_hours_per_subject) * len(study_days)
    remaining = target_hours - total_hours
    return df_detail, df_summary, remaining, target_hours

new_student = pd.DataFrame({
    "Available_Hours_Per_Day": [4],
    "Free_Days_Per_Week": [5],
    "Weak_Subjects_Count": [2]
})

predicted_hours = [2, 3, 4, 2, 1, 2]
subject_names = ["Math", "Physics", "CS", "Chemistry", "English", "Biology"]
available_hours_per_day = int(new_student.loc[0, "Available_Hours_Per_Day"])
free_days_per_week = int(new_student.loc[0, "Free_Days_Per_Week"])
weak_subjects = int(new_student.loc[0, "Weak_Subjects_Count"])

df_detail, df_summary, remaining, target_hours = generate_weekly_schedule(
    predicted_hours_per_subject=predicted_hours,
    subject_names=subject_names,
    available_hours_per_day=available_hours_per_day,
    free_days_per_week=free_days_per_week,
    weak_subjects=weak_subjects,
    weak_extra=0.5,
    session_granularity=0.5
)

df_summary.to_csv("weekly_schedule_summary.csv", index=False)
df_detail.to_csv("weekly_schedule_detailed.csv", index=False)


C:\Users\astle\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200

Epoch 1: val_loss improved from None to 0.72472, saving model to study_plan_model.h5


22/22 - 7s - 326ms/step - loss: 0.8217 - mae: 0.6948 - val_loss: 0.7247 - val_mae: 0.7175 - learning_rate: 0.0010
Epoch 2/200

Epoch 2: val_loss improved from 0.72472 to 0.45925, saving model to study_plan_model.h5


22/22 - 1s - 26ms/step - loss: 0.5093 - mae: 0.5603 - val_loss: 0.4592 - val_mae: 0.5091 - learning_rate: 0.0010
Epoch 3/200

Epoch 3: val_loss improved from 0.45925 to 0.31152, saving model to study_plan_model.h5


22/22 - 1s - 23ms/step - loss: 0.3476 - mae: 0.4659 - val_loss: 0.3115 - val_mae: 0.3853 - learning_rate: 0.0010
Epoch 4/200

Epoch 4: val_loss improved from 0.31152 to 0.22546, saving model to study_plan_model.h5


...

22/22 - 1s - 39ms/step - loss: 0.0515 - mae: 0.1688 - val_loss: 0.0120 - val_mae: 0.0847 - learning_rate: 0.0010
Epoch 44/200

Epoch 44: val_loss did not improve from 0.01204
22/22 - 0s - 19ms/step - loss: 0.0500 - mae: 0.1697 - val_loss: 0.0228 - val_mae: 0.1076 - learning_rate: 0.0010
Epoch 45/200

Epoch 45: val_loss did not improve from 0.01204
22/22 - 1s - 29ms/step - loss: 0.0416 - mae: 0.1553 - val_loss: 0.0137 - val_mae: 0.0875 - learning_rate: 0.0010
Epoch 46/200

Epoch 46: val_loss did not improve from 0.01204
22/22 - 0s - 16ms/step - loss: 0.0392 - mae: 0.1505 - val_loss: 0.0131 - val_mae: 0.0872 - learning_rate: 0.0010
Epoch 47/200

Epoch 47: val_loss did not improve from 0.01204
22/22 - 1s - 29ms/step - loss: 0.0412 - mae: 0.1545 - val_loss: 0.0143 - val_mae: 0.0939 - learning_rate: 0.0010
Epoch 48/200

Epoch 48: val_loss did not improve from 0.01204
22/22 - 1s - 29ms/step - loss: 0.0390 - mae: 0.1501 - val_loss: 0.0131 - val_mae: 0.0808 - learning_rate: 0.0010
Epoch 49/200

22/22 - 1s - 34ms/step - loss: 0.0288 - mae: 0.1317 - val_loss: 0.0089 - val_mae: 0.0716 - learning_rate: 0.0010
Epoch 52/200

Epoch 52: val_loss did not improve from 0.00888
22/22 - 1s - 25ms/step - loss: 0.0335 - mae: 0.1437 - val_loss: 0.0149 - val_mae: 0.0853 - learning_rate: 0.0010
Epoch 53/200

Epoch 53: val_loss did not improve from 0.00888
22/22 - 0s - 15ms/step - loss: 0.0354 - mae: 0.1404 - val_loss: 0.0109 - val_mae: 0.0795 - learning_rate: 0.0010
Epoch 54/200

Epoch 54: val_loss did not improve from 0.00888
22/22 - 0s - 16ms/step - loss: 0.0384 - mae: 0.1470 - val_loss: 0.0218 - val_mae: 0.0973 - learning_rate: 0.0010
Epoch 55/200

Epoch 55: val_loss did not improve from 0.00888
22/22 - 1s - 29ms/step - loss: 0.0354 - mae: 0.1403 - val_loss: 0.0113 - val_mae: 0.0814 - learning_rate: 0.0010
Epoch 56/200

Epoch 56: val_loss did not improve from 0.00888
22/22 - 0s - 15ms/step - loss: 0.0333 - mae: 0.1369 - val_loss: 0.0101 - val_mae: 0.0763 - learning_rate: 0.0010
Epoch 57/200

22/22 - 0s - 22ms/step - loss: 0.0274 - mae: 0.1251 - val_loss: 0.0082 - val_mae: 0.0716 - learning_rate: 2.5000e-04
Epoch 70/200

Epoch 70: val_loss did not improve from 0.00821
22/22 - 1s - 25ms/step - loss: 0.0296 - mae: 0.1327 - val_loss: 0.0098 - val_mae: 0.0769 - learning_rate: 2.5000e-04
Epoch 71/200

Epoch 71: val_loss did not improve from 0.00821
22/22 - 0s - 15ms/step - loss: 0.0296 - mae: 0.1269 - val_loss: 0.0117 - val_mae: 0.0808 - learning_rate: 2.5000e-04
Epoch 72/200

Epoch 72: val_loss did not improve from 0.00821
22/22 - 1s - 29ms/step - loss: 0.0272 - mae: 0.1238 - val_loss: 0.0091 - val_mae: 0.0742 - learning_rate: 2.5000e-04
Epoch 73/200

Epoch 73: val_loss did not improve from 0.00821
22/22 - 0s - 17ms/step - loss: 0.0314 - mae: 0.1318 - val_loss: 0.0116 - val_mae: 0.0815 - learning_rate: 2.5000e-04
Epoch 74/200

Epoch 74: val_loss did not improve from 0.00821
22/22 - 0s - 17ms/step - loss: 0.0334 - mae: 0.1349 - val_loss: 0.0094 - val_mae: 0.0758 - learning_rate:

22/22 - 1s - 36ms/step - loss: 0.0294 - mae: 0.1274 - val_loss: 0.0082 - val_mae: 0.0714 - learning_rate: 1.2500e-04
Epoch 79/200

Epoch 79: val_loss did not improve from 0.00818
22/22 - 1s - 23ms/step - loss: 0.0298 - mae: 0.1305 - val_loss: 0.0092 - val_mae: 0.0742 - learning_rate: 1.2500e-04
Epoch 80/200

Epoch 80: val_loss did not improve from 0.00818
22/22 - 1s - 30ms/step - loss: 0.0286 - mae: 0.1294 - val_loss: 0.0089 - val_mae: 0.0726 - learning_rate: 1.2500e-04
Epoch 81/200

Epoch 81: val_loss did not improve from 0.00818
22/22 - 1s - 32ms/step - loss: 0.0290 - mae: 0.1291 - val_loss: 0.0093 - val_mae: 0.0740 - learning_rate: 1.2500e-04
Epoch 82/200

Epoch 82: val_loss did not improve from 0.00818
22/22 - 1s - 29ms/step - loss: 0.0310 - mae: 0.1307 - val_loss: 0.0088 - val_mae: 0.0725 - learning_rate: 1.2500e-04
Epoch 83/200

Epoch 83: val_loss did not improve from 0.00818
22/22 - 0s - 18ms/step - loss: 0.0312 - mae: 0.1318 - val_loss: 0.0098 - val_mae: 0.0760 - learning_rate:

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step
